In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from result_saver import SaverProvider

provider = SaverProvider()

In [22]:
from Scratch import metadata_loader

DEVICE = 'ibm_sherbrooke'
DISTANCE = 10
ROUNDS = 10
LOGICAL = 0

md = metadata_loader(True, True)
md = md[md['meas_level'] == 0]
md = md[md['distance']==DISTANCE]
md = md[md['rounds']==ROUNDS]
md = md[md['logical']==LOGICAL]

md = md[md['notebook_name']=='mean_pos_sherbrooke']

md

,creation_date,notebook_name,backend_name,job_id,tags,meas_level,shots,tags_xp,rep_delay,sampled_state,...,optimization_level,code,distance,rounds,logical,layout,descr,num_CNOTS_layers,num_CNOT_layers,resets
2620,2024-03-10 20:58:28.709000+01:00,mean_pos_sherbrooke,ibm_sherbrooke,cqq11s7jx32g008542tg,"[50*1e6/d*T, RepCode Mean Positions]",0.0,5555.0,NaN,0.0005,NaN,...,NaN,RepetitionCodeCircuit,10.0,10,0,NaN,Mean positions,NaN,NaN,False
2619,2024-03-10 20:57:46.022000+01:00,mean_pos_sherbrooke,ibm_sherbrooke,cqq11edjx32g008542s0,"[RepCode Mean Positions, 10*1e6/d*T]",0.0,1111.0,NaN,0.0005,NaN,...,NaN,RepetitionCodeCircuit,10.0,10,0,NaN,Mean positions,NaN,NaN,False
2618,2024-03-10 20:57:39.190000+01:00,mean_pos_sherbrooke,ibm_sherbrooke,cqq11cn4td0g008cp2sg,"[RepCode Mean Positions, 10*1e6/d*T]",0.0,1111.0,NaN,0.0005,NaN,...,NaN,RepetitionCodeCircuit,10.0,10,0,NaN,Mean positions,NaN,NaN,False
2617,2024-03-10 20:56:36.833000+01:00,mean_pos_sherbrooke,ibm_sherbrooke,cqq10x3jx32g008542r0,"[RepCode Mean Positions, 1e6/d*T]",0.0,111.0,NaN,0.0005,NaN,...,NaN,RepetitionCodeCircuit,10.0,10,0,NaN,Mean positions,NaN,NaN,False
2616,2024-03-10 20:56:30.367000+01:00,mean_pos_sherbrooke,ibm_sherbrooke,cqq10vkv78p00084kpk0,"[RepCode Mean Positions, 1e6/d*T]",0.0,111.0,NaN,0.0005,NaN,...,NaN,RepetitionCodeCircuit,10.0,10,0,NaN,Mean positions,NaN,NaN,False
2615,2024-03-10 20:56:12.756000+01:00,mean_pos_sherbrooke,ibm_sherbrooke,cqq10q24td0g008cp2ng,"[RepCode Mean Positions, 1e6/d*T]",0.0,111.0,NaN,0.0005,NaN,...,NaN,RepetitionCodeCircuit,10.0,10,0,NaN,Mean positions,NaN,NaN,False
2614,2024-03-10 20:56:05.331000+01:00,mean_pos_sherbrooke,ibm_sherbrooke,cqq10na417q0008ysdp0,"[RepCode Mean Positions, 1e6/d*T]",0.0,111.0,NaN,0.0005,NaN,...,NaN,RepetitionCodeCircuit,10.0,10,0,NaN,Mean positions,NaN,NaN,False
2613,2024-03-10 20:55:59.348000+01:00,mean_pos_sherbrooke,ibm_sherbrooke,cqq10ktv78p00084kph0,"[RepCode Mean Positions, 1e6/d*T]",0.0,111.0,NaN,0.0005,NaN,...,NaN,RepetitionCodeCircuit,10.0,10,0,NaN,Mean positions,NaN,NaN,False
2612,2024-03-10 20:55:53.329000+01:00,mean_pos_sherbrooke,ibm_sherbrooke,cqq10ja4td0g008cp2mg,"[RepCode Mean Positions, 1e6/d*T]",0.0,111.0,NaN,0.0005,NaN,...,NaN,RepetitionCodeCircuit,10.0,10,0,NaN,Mean positions,NaN,NaN,False
2611,2024-03-10 20:55:45.511000+01:00,mean_pos_sherbrooke,ibm_sherbrooke,cqq10ga267h0008fj8r0,"[RepCode Mean Positions, 1e6/d*T]",0.0,111.0,NaN,0.0005,NaN,...,NaN,RepetitionCodeCircuit,10.0,10,0,NaN,Mean positions,NaN,NaN,False


# Initialize the decoder

In [23]:
from soft_info import get_noise_dict_from_backend, get_avgs_from_dict

noise_dict = get_noise_dict_from_backend(provider, DEVICE, date = md.iloc[0]['creation_date'])

In [24]:
from soft_info import get_repcode_IQ_map, counts_to_det_syndr
import stim
import pymatching
from tqdm import tqdm
import numpy as np

num_errors_per_job = []
num_shots_per_job = []

for job_id in tqdm(md['job_id']):
    num_shots_per_job_ = 0
    job = provider.retrieve_job(job_id)
    layouts = job.initial_layouts()

    num_errors_per_layout = []
    for idx, layout in enumerate(layouts):
        
        # Get the layout + noise
        layout_des = job.deserialize_layout(layout)    
        layout = list(layout_des['link_qubit'].values()) + list(layout_des['code_qubit'].values())
        qubit_mapping = get_repcode_IQ_map(layout, synd_rounds=3)
        avgs = get_avgs_from_dict(noise_dict, layout)

        # Initialize the stim model
        circuit = stim.Circuit.generated("repetition_code:memory",
                                distance=DISTANCE,
                                rounds=ROUNDS,
                                after_clifford_depolarization=avgs["two_gate"], #two-qubit-fidelity,
                                after_reset_flip_probability = 0, #if not _RESETS else avgs["readout"], #reset error
                                before_measure_flip_probability=avgs["readout"], #measurement error,
                                before_round_data_depolarization=avgs["idle"]) #idle error)
        model = circuit.detector_error_model(decompose_errors=False)
        matching = pymatching.Matching.from_detector_error_model(model)

        # Decode the counts
        counts = job.result().get_counts(idx)
        num_errors = 0
        for count_key, shots in counts.items():

            array_processed_string = counts_to_det_syndr(count_key, _resets=False, verbose=False)
            predicted_observables = matching.decode(array_processed_string)
            actual_observables = [(int(count_key[0])+0)%2]

            if actual_observables != predicted_observables:
                num_errors += shots
        
        num_errors_per_layout.append(num_errors)
        num_shots = sum(counts.values())
        num_shots_per_job_ += num_shots
    num_shots_per_job.append(num_shots_per_job_)
    num_errors_per_job.append(np.sum(num_errors_per_layout))

    print(f"Job {job_id} had {num_errors_per_layout} errors per layout out of total {num_shots_per_job} shots")




  8%|▊         | 1/13 [00:35<07:04, 35.37s/it]

Job cqq11s7jx32g008542tg had [39, 22, 21, 31, 10, 21, 46, 22, 11, 19, 9, 18, 2, 47, 9, 34, 3, 35, 8, 31, 17, 137, 27, 132, 46, 284, 64, 388, 149, 399, 296, 587, 243, 247, 101, 178, 30, 82, 27, 31, 25, 53, 37, 44, 70, 28, 13, 18, 19, 40, 21, 57, 39, 64, 34, 64, 41, 183, 87, 46, 113, 64, 158, 70, 312, 53, 368, 53, 291, 113, 206, 100, 295, 68, 165, 101, 98, 81, 36, 44, 36, 36, 13, 36, 11, 30, 3, 15, 2, 31] errors per layout out of total [499950] shots


 15%|█▌        | 2/13 [00:42<03:27, 18.85s/it]

Job cqq11edjx32g008542s0 had [5, 9, 5, 3, 5, 5, 4, 3, 2, 3, 2, 6, 2, 14, 3, 8, 0, 5, 2, 5, 1, 21, 6, 20, 6, 63, 14, 72, 30, 81, 48, 118, 53, 39, 11, 33, 5, 10, 7, 4, 3, 7, 3, 8, 16, 3, 7, 5, 2, 8, 2, 15, 3, 18, 10, 11, 3, 38, 19, 21, 29, 17, 33, 11, 56, 11, 65, 4, 56, 19, 38, 18, 69, 14, 36, 20, 7, 16, 8, 13, 7, 10, 2, 11, 1, 3, 2, 3, 1, 4] errors per layout out of total [499950, 99990] shots


 23%|██▎       | 3/13 [00:49<02:15, 13.56s/it]

...

Job cqq10ga267h0008fj8r0 had [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 3, 0, 1, 0, 1, 0, 2, 1, 3, 0, 4, 1, 12, 0, 10, 1, 16, 7, 13, 3, 4, 0, 3, 0, 0, 1, 0, 0, 3, 1, 3, 1, 1, 0, 0, 0, 1, 0, 2, 0, 1, 0, 2, 0, 4, 0, 1, 4, 3, 3, 0, 3, 2, 5, 2, 4, 1, 6, 1, 5, 1, 1, 0, 1, 2, 1, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0] errors per layout out of total [499950, 99990, 99990, 9990, 9990, 9990, 9990, 9990, 9990, 9990] shots
Job cqq10es267h0008fj8qg had [0, 0, 0, 2, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 2, 0, 1, 0, 0, 0, 2, 1, 4, 2, 9, 3, 11, 5, 13, 8, 17, 3, 5, 0, 3, 1, 0, 1, 0, 0, 0, 1, 1, 2, 2, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 2, 6, 2, 1, 2, 0, 3, 0, 1, 1, 6, 0, 7, 2, 3, 1, 4, 1, 5, 3, 0, 1, 0, 1, 1, 0, 0, 2, 1, 0, 1, 0, 0, 4] errors per layout out of total [499950, 99990, 99990, 9990, 9990, 9990, 9990, 9990, 9990, 9990, 9990] shots


 92%|█████████▏| 12/13 [00:57<00:01,  1.16s/it]

Job cqq10d9417q0008ysdmg had [1, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 2, 1, 2, 0, 3, 0, 4, 0, 3, 2, 13, 2, 9, 7, 7, 6, 17, 9, 9, 1, 2, 0, 3, 1, 0, 0, 0, 0, 1, 1, 2, 0, 0, 0, 1, 0, 1, 0, 2, 1, 1, 0, 3, 2, 2, 2, 3, 1, 1, 7, 1, 10, 2, 7, 5, 5, 1, 7, 1, 3, 1, 4, 2, 0, 1, 4, 1, 0, 0, 0, 2, 0, 0, 0, 2] errors per layout out of total [499950, 99990, 99990, 9990, 9990, 9990, 9990, 9990, 9990, 9990, 9990, 9990] shots


100%|██████████| 13/13 [00:58<00:00,  4.47s/it]

Job cqq10bsjx32g008542ng had [0, 1, 1, 0, 1, 3, 2, 1, 2, 0, 1, 0, 0, 1, 0, 2, 0, 1, 1, 2, 0, 6, 1, 4, 1, 10, 1, 9, 6, 9, 7, 13, 7, 8, 5, 4, 1, 2, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 1, 0, 1, 2, 1, 1, 4, 2, 7, 0, 9, 2, 3, 2, 5, 3, 10, 0, 8, 1, 4, 1, 2, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0] errors per layout out of total [499950, 99990, 99990, 9990, 9990, 9990, 9990, 9990, 9990, 9990, 9990, 9990, 9990] shots


In [25]:
# Calculate the error rate
print('num_errors_per_job', num_errors_per_job)
print('num_shots_per_job', num_shots_per_job)
print('sum(num_errors_per_job)', np.sum(num_errors_per_job))
print('sum(num_shots_per_job)', np.sum(num_shots_per_job))

error_rate = np.sum(num_errors_per_job) / np.sum(num_shots_per_job)

print(f"The error rate is {error_rate:.2e}")

num_errors_per_job [7888, 1519, 1675, 159, 188, 177, 151, 165, 160, 156, 166, 200, 194]
num_shots_per_job [499950, 99990, 99990, 9990, 9990, 9990, 9990, 9990, 9990, 9990, 9990, 9990, 9990]
sum(num_errors_per_job) 12798
sum(num_shots_per_job) 799830
The error rate is 1.60e-02
